In [1]:
import numpy as np
import pandas as pd
import os
import json
from dateutil import parser
from zipfile import ZipFile


[1, 2, 3]

In [3]:
data = {
#     "Benchmark": ["X-bit","X-bit","X-bit","X-bit"],
#     "R": [4,4,4,4],
#     "I": [1,2,3,4],
#     "Dp": [200,200,200,200],
#     "Ld": [40,40,40,40],
#     "Nb": [5,5,5,5]
       
       }

macro = pd.DataFrame(data, columns=["Benchmark", "R", "I", "Dp", "Ld", "Nb"])
macro = macro.astype(
    {'Benchmark': 'str',
     'R': 'int',
     'I': 'int',
     'Dp': 'int',
     'Ld': 'int',
     'Nb': 'int'}
)
macro

,Benchmark,R,I,Dp,Ld,Nb


In [4]:
data = {
    
}

exp = pd.DataFrame(data, columns=["macroExperiment", "CARule", "StartTime"])
# for i in range(0,4):
#     for ca_rule in CA_rule_order:
#         exp = exp.append({"macroExperiement":i, "CARule": ca_rule}, ignore_index=True
# )
    
exp
exp = exp.astype(
    {'macroExperiment': 'int',
     'CARule': 'int',
     'StartTime': 'datetime64[ns]'}
)
exp

,macroExperiment,CARule,StartTime


In [5]:
data = {}
run = pd.DataFrame(data, columns=["Experiment", "Avg", "RandomMapping"])
run

,Experiment,Avg,RandomMapping


In [6]:
%%time

def paramsToDict(params):
    dictP = {}
    for pa in params:
        arrayP = pa.split("=")
        if len(arrayP) != 2:
            arrayP = ["started at", "".join(arrayP[0].split(":")[1:5]).replace("\n", "")]
#         print(arrayP)
        dictP[arrayP[0].strip()] = arrayP[1]
    return dictP

from zipfile import ZipFile
directory = "import"
for zipname in os.listdir(directory):
    with ZipFile(os.path.join(directory, zipname), "r") as zip:
        for filename in zip.namelist():
            with zip.open(filename) as file:
                lines = file.readlines()
                params = (lines[0].decode("utf-8")).split(",")
                dictP = paramsToDict(params)
#                 print(dictP)
                ex = {
                    "Nb": int(dictP.get("bits", 5)),
                    "R" : int(dictP.get("r", -1)),
                    "I" : int(dictP.get("itr", -1)),
                    "Ld": int(dictP.get("r width")) if dictP.get("r width") != None else int(dictP.get("r total width", -1)) / int(dictP.get("r", -1)) ,
                    "Dp": int(dictP.get("distractor period", -1)),
                    "CA": int(dictP.get("CA rule", -1)),
                    "Nruns": int(dictP.get("number of runs", -1)),
                    "StartTime": parser.parse(dictP.get("started at", -1)),
                }
                exIndex = exp.index.size
                macroExp = macro[(macro["Nb"]==ex["Nb"])
                         & (macro["R"]==ex["R"])
                         & (macro["I"]==ex["I"])
                         & (macro["Ld"]==ex["Ld"])
                         & (macro["Dp"]==ex["Dp"])
                            ]
                macroIndex = None
                if(macroExp.empty):
                    macroIndex = macro.index.size
                    macro = macro.append({"Benchmark":"X-Bit", "Nb": ex["Nb"],"R": ex["R"],"I": ex["I"],"Ld": ex["Ld"],"Dp": ex["Dp"]}, ignore_index=True)
                else:
                    macroIndex = macroExp.index.max()
                exp =exp.append({"macroExperiment": macroIndex, "CARule":ex["CA"], "StartTime": ex["StartTime"]}, ignore_index=True)

                runs = lines[2:ex["Nruns"]+2]
                for inputRun in runs:
                    inputRun = inputRun.decode("utf-8")
                    Score = float(inputRun.split("\t")[0])
                    map = json.loads((inputRun.split("\t")[1]).replace("\n", ""))
                    run = run.append({"Experiment": exIndex, "Avg":Score, "RandomMapping":map}, ignore_index=True)

    

Wall time: 47min 36s


In [7]:
macro

,Benchmark,R,I,Dp,Ld,Nb
0,X-Bit,4,2,100,40.000000,5
1,X-Bit,4,2,300,40.000000,5
2,X-Bit,4,2,4000,40.000000,5
3,X-Bit,4,2,50,40.000000,5
4,X-Bit,1,1,195,35.000000,5
...,...,...,...,...,...,...
353,X-Bit,3,2,200,55.333333,5
354,X-Bit,3,2,200,55.666667,5
355,X-Bit,3,2,200,56.000000,5
356,X-Bit,3,2,200,56.333333,5


In [8]:
exp

,macroExperiment,CARule,StartTime
0,0,0,2021-03-28 15:32:43.760279
1,0,1,2021-03-28 16:15:39.122867
2,0,2,2021-03-28 16:58:00.030187
3,0,3,2021-03-28 17:40:20.498386
4,0,4,2021-03-28 18:23:19.159263
...,...,...,...
3808,357,251,2021-01-14 08:55:28.789110
3809,357,252,2021-01-14 09:43:32.486915
3810,357,253,2021-01-14 10:31:34.239061
3811,357,254,2021-01-14 11:19:37.569620


In [9]:
run

,Experiment,Avg,RandomMapping
0,0,0.954545,"[30, 8, 1, 35, 61, 49, 40, 45, 93, 101, 94, 11..."
1,0,0.954545,"[24, 13, 38, 5, 61, 49, 40, 69, 94, 100, 111, ..."
2,0,0.954545,"[6, 20, 36, 11, 46, 75, 43, 52, 91, 117, 103, ..."
3,0,0.954545,"[13, 32, 6, 39, 64, 46, 45, 71, 109, 115, 80, ..."
4,0,0.954545,"[8, 21, 29, 10, 52, 45, 51, 54, 115, 111, 85, ..."
...,...,...,...
381295,3812,0.976190,"[11, 30, 6, 3, 44, 70, 49, 76, 101, 91, 106, 9..."
381296,3812,0.976190,"[4, 37, 0, 3, 47, 43, 45, 64, 118, 99, 106, 94..."
381297,3812,0.976190,"[27, 22, 26, 24, 73, 79, 54, 49, 83, 118, 109,..."
381298,3812,0.976190,"[34, 24, 35, 1, 79, 66, 54, 69, 92, 84, 105, 1..."


In [37]:
%%time
macro.to_csv("macro.csv")
exp.to_csv("exp.csv")
run.to_csv("run.csv")

Wall time: 1.27 s


In [38]:
%%time
macro = pd.read_csv('macro.csv', index_col=0)  
exp = pd.read_csv('exp.csv', index_col=0)  
run = pd.read_csv('run.csv', index_col=0)  

Wall time: 445 ms


In [28]:
count = 0
def metrics(row):
    mn = (row.Dp.iloc[0]+row.Nb.iloc[0])/(row.Dp.iloc[0]+(2*row.Nb.iloc[0]))
    count = row.Avg.count()
    mean = row.Avg.mean()
    perf = row.Avg[row.Avg==1.0].count()
    ob = pd.Series(data = {"W.Avg." : (mean - mn)/(1-mn), "mean": mean, "perf": perf, "count":count})
    return ob


mega = pd.merge(macro, pd.merge(exp, run, right_on="Experiment", left_index=True), right_on="macroExperiment", left_index=True)

megaG = mega.groupby(["macroExperiment", "I", "Nb", "Dp", "CARule", "Ld"])

megaG.apply(metrics)


W.Avg.      mean  perf  count
macroExperiment I Nb Dp  CARule Ld                                       
0               2 5  100 0      40.0 -2.198242e-14  0.954545   0.0  100.0
                         1      40.0  5.686250e-01  0.980392   0.0  100.0
                         2      40.0  5.000000e-01  0.977273   0.0  100.0
                         3      40.0  9.624375e-01  0.998293   7.0  100.0
                         4      40.0  5.200000e-01  0.978182   0.0  100.0
...                                            ...       ...   ...    ...
357             2 5  200 251    40.0  4.196643e-14  0.976190   0.0  100.0
                         252    40.0  4.196643e-14  0.976190   0.0  100.0
                         253    40.0  4.196643e-14  0.976190   0.0  100.0
                         254    40.0  4.196643e-14  0.976190   0.0  100.0
                         255    40.0  4.196643e-14  0.976190   0.0  100.0

[3442 rows x 4 columns]

In [29]:
megaGC = megaG.count()

In [ ]:
clean = megaGC[megaGC != 100].dropna()

In [ ]:
run.duplicated()

NameError: name 'run' is not defined